### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

In [ ]:
n_cycles = 1_000_000

In [ ]:
start.record()
torch.cuda._sleep(n_cycles)
end.record()

In [ ]:
cycle_per_ms = n_cycles / start.elapsed_time(end)

In [ ]:
def cuda_sleep(seconds)

In [ ]:
received queue, job queue, output queue, syncronization queue

In [ ]:
pool thread, job selector, worker thread

In [ ]:
partition2(microbatch1)

In [ ]:
int zero() {
    return 0;
}

In [ ]:
#include <iostream>

In [ ]:
class Book() {
    public:
        std::string title;
        std::string author;
}

In [ ]:
x.repeat((3, 2))

In [ ]:
min(clock_idx+1, n_partitions)

In [ ]:
notification service

In [ ]:
class ParallelContext:
    def init_rpc_workers(self, host, port):
        if self.pipeline_parallel_size > 1:
            rank = self.get_local_rank(ParallelMode.PIPELINE)
            world_size = self.get_world_size(ParallelMode.PIPELINE)
            init_method = f"rpc://{host}:{port}"
            
            options = rpc.RpcBackendOptions(
                init_method=init_method
            )
            
            if torch.cuda.is_available():
                ranks = self.get_ranks_in_group(ParallelMode.PIPELINE)
                
                device_mapping = {
                    rank: WORKER_NAME.format(rank)
                    for rank in ranks
                }
                
                for other in ranks:
                    if other == rank:
                        continue
                    options.set_device(WORKER_NAME.format(rank), rank: other)
            
            rpc.init_rpc(
                name=WORKER_NAME.format(rank),
                world_size=world_size,
                rpc_backend_options=options
            )

In [5]:
class _P2P:
    def _send_metadata(self, data, dst, parallel_context, parallel_mode):
        group = parallel_context.get_group(ParallelMode.PIPELINE)
        
        dtype = torch.tensor(DTYPE_TO_ID[data.dtype])
        dist.send(dtype, dst=dst, group=group)
        
        shape = torch.tensor(data.shape.to_list())
        dist.send(shape, dst=dst, group=group)
        
        requires_grad = torch.tensor(1 if data.requires_grad == True else 0)
        dist.send(requires_grad, dst=dst, group=group)
    
    def send(self, data, dst, parallel_context, parallel_mode):
        group = parallel_context.get_group(ParallelMode.PIPELINE)
        self._send_metadata(data, dst, parallle_context, parallel_mode)
        dist.send(data, dst, group=group)

In [ ]:
def send(data, src_rank, dst_rank, parallel_context, parallel_mode=ParallelMode.PIPELINE):
    rank = parallel_context.get_local_rank(parallel_mode)
    if rank == src_rank:
        _P2P().send(data, dst_rank, parallel_context, parallel_mode)

In [ ]:
class VocabParallelCrossEntropy

In [6]:
from transformer_lens.utils import get_act_name

In [ ]:
mlp_names = [get_act_name("mlp_out", layer_idx) for layer_idx in range(n_layers)]
attn_names = [get_act_name("attn_out", layer_idx) for layer_idx in range(n_layers)]

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
input_components = torch.tensor([cache["embed"], cache["pos_embed"]])

In [ ]:
for attn_name, mlp_name in zip(mlp_names, attn_names):
    input_components = torch.cat([
        input_components,
        cache[attn_name],
        cache[mlp_name]
    ])

In [ ]:
final_ln = cache["final_ln"]
pre_ln = final_ln @ final_ln_coefs.T

In [ ]:
W_U = model.W_U
pre_ln_dir = pre_ln @ (W_U[:, 0], W_U[:, 1])

In [7]:
contributions = []

In [ ]:
_, cache = model.run_with_cache(clean_tokens)

In [8]:
layer_idx, head_idx = 9, 9

In [ ]:
attn_output = cache["z", layer_idx][:, : head_idx] @ model.W_O[layer_idx, head_idx]

In [ ]:
W_E = model.W_E
io_embeddings = model.W_E[:, io_tokens]
s_embeddings = model.W_E[:, s_tokens]

In [ ]:
projection_in_io_dir = attn_output @ io_embeddings
projection_in_s_dir = attn_output @ s_embeddings

In [ ]:
attn_probs = cache["attn_prob", layer_idx][:, :, head_idx]
attn_from_end_to_io = attn_probs[:, -1, io_idxs]
attn_from_end_to_s = attn_probs[:, -1, s_idxs

In [ ]:
step 1: logits = final_resid @ W_U
step 2: final_resid = embed + pos_embed + layer0 + layer1
step 3: logits = [embed + pos_embed + layer0 + layer1] @ W_U
step 4: logits = embed @ W_U + pos_embed @ W_U + layer0 @ W_U + layer1 @ W_U

In [9]:
from einops import einsum

In [ ]:
interference = einsum(
    W, W,
    "b f1 d, b f2 d -> b f1 f2"
)

In [10]:
layer_idx, head_idx = 6, 9

In [ ]:
W_V = model.W_V[layer_idx, head_idx]
W_O = model.W_O[layer_idx, head_idx]

In [ ]:
W_E = model.W_E
W_U = model.W_U

In [ ]:
W_OV = W_V @ W_O

In [ ]:
W_E = model.W_E
W_Q = model.W_Q[1, 4]
W_K = model.W_K[1, 4]
W_O = model.W_O[0, 7]
W_V = model.W_V[0, 7]

In [ ]:
Q = W_E @ W_Q
K = W_E @ W_V @ W_O @ W_K

In [ ]:
int zero() {
    return 0;
}

In [ ]:
received queue, job queues, output queues, syncronization queue

In [ ]:
step 1: 0 send ready
step 2: other received and send back
step 3: 

In [ ]:
scatter, reduce

In [ ]:
F.layer_norm(input, normalized_shape=(5,))

In [11]:
from einops import rearrange

In [ ]:
class VocabParallelCrossEntropy(torch.autograd.Function):
    @staticmethod
    def forward(ctx, parallel_logits, targets, parallel_context):
        def get_vocab_range(parallel_logits, parallel_context):
            partition_size = parallel_logits.shape[-1]
            rank = parallel_context.get_local_rank(ParallelMode.TENSOR)
            start_idx = rank*partition_size
            end_idx = start_idx+partition_size
            return start_idx, end_idx
        
        def get_predicted_logits(parallel_logits, targets):
            start_idx, end_idx = get_vocab_range(parallel_logits, parallel_context)
            target_mask = (targets < start_idx) | (targets >= end_idx)
            masked_targets = targets = targets.clone() - start_idx
            masked_targets[target_mask] = 0
            
            parallel_logits = rearrange(
                parallel_logits,
                "batch_size seq_len vocab_size -> (batch_size seq_len) vocab_size"
            )
            masked_target_1d = rearrange(
                masked_targets,
                "batch_size seq_len -> (batch_size seq_len)"
            )
            predicted_logits = parallel_logits[
                torch.arange(masked_target.size(0)),
                masked_target_1d
            ]
            predicted_logits = all_reduce

In [ ]:
step 1: define a mapping
step 2: mapping
step 3: send idx
step 4: 

In [ ]:
class TensorParallelGroupInitialize(ProcessGroup):
    def init_dist_group(self):
        num_tensor_parallel_groups = self.world_size // self.tensor_parallel_size
        local_rank = None
        local_world_size = None
        process_group = None
        ranks_in_group = None
        parallel_mode = ParallelMode.TENSOR
        
        for i in range(num_tensor_parallel_groups):
            ranks = list(range(
                i*self.tensor_parallel_size, (i+1)*self.tensor_parallel_size
            ))
            process_group = dist.new_group(ranks=ranks)
            
            if self.rank in ranks:
                local_rank = ranks.index(self.rank)
                local_world_size = len(ranks)
                ranks_in_group = ranks
        
        return {
            ""
        }

In [ ]:
class Pipeline:
    def __init__(
        self,
        microbatches,
        partitions,
        devices,
        scheduler=DetermisticScheduler()
    ):
        self.microbatches = microbatches
        self.partitions = partitions
        self.devices = devices
        
    def fit(self):
        n_microbatches = len(self.microbatches)
        n_partitions = len(self.partitions)
        
        with spawn_workers(self.devices) as (in_queues, out_queues):
            for schedule in self.scheduler(n_microbatches, n_partitions):
                self._compute(in_queues, out_queues, schedule)
    
    def _compute(self, in_queues, out_queues, schedule):
        for microbatch_idx, partition_idx in schedule:
            batch = self.microbatches[microbatch_idx]
            
            def compute_func():
                def inner():
                    return partitions[partition_idx](batch)
                return inner
            
            task = Task(compute=compute_func)
            in_queues[partition_idx].put(task)
        
        for microbatch_idx, partition_idx in schedule:
            output = out_queues.get(partition_idx)
            batches[microbatch_idx].put(output)

In [ ]:
step 1: pack data + metadata
step 2: invoke rpc + data
step 3: receive
step 4: 

In [ ]:
torch.roll(x, shifts=1, dim=0)

In [ ]:
step 1: wait input
step 2: get input
step 3: construct task
step 4: put 
step 5: wait
step 6: get
step 7: put

In [ ]:
step 1: determine t

In [12]:
layer_idx, head_idx = 6, 9

In [ ]:
W_E = model.W_E
W_U = model.W_U

W_V = model.W_V[layer_idx, head_idx]
W_O = model.W_O[layer_idx, head_idx]

In [ ]:
OV_circuit = W_V @ W_O

In [ ]:
full_OV_circuit = W_E @ OV_circuit @ W_U

In [13]:
layer_idx, head_idx = 0, 2

In [ ]:
_, cache = model.run_with_cache(clean_tokens)

In [ ]:
cache["z", layer_idx][:, :, head_idx] @ model.W_O[layer_idx, head_idx]

In [ ]:
W_E @ W_OV^{0, 7} @ W_QK^{T}{1, 4} @ W_U

In [ ]:
W_E = model.W_E
W_E[:, tokens]

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [14]:
from transformer_lens.utils import get_act_name

In [ ]:
hook_name = get_act_name("resid_post", layer_idx)

In [15]:
def intervene_resid(resid, hook, scale, feature):
    normalized_feature /= feature.norm()
    feature_projection = resid[0, position] @ normalized_feature
    resid[0, position] -= scale*feature_projection*normalized_feature
    return resid

In [16]:
from functools import partial

In [ ]:
hook_func = partial(
    intervene_resid,
    scale=scale_factor,
    feature=target_feature
)

In [ ]:
intervened_logits = model.run_with_hooks(
    past_moves,
    fwd_hooks([hook_name, hook_func])
)

In [ ]:
intervened_logits = interven

In [ ]:
embed = model.W_E
unembed = model.W_U

mlp0 = model.blocks[0].mlp
ln1 = model.blocks[0].ln1

final_ln = model.final_ln

In [ ]:
name_tokens = model.to_tokens(name_tokens, prepend_bos=True)
embeddings = embed(name_tokens)

In [ ]:
resid_after_mlp0 = embeddings + mlp0(ln1(embeddings))

In [17]:
def resid2logits(resid):
    return unembed(final_ln(resid))

In [ ]:
for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        W_OV = model.W_V[layer_idx, head_idx] @ W_O[layer_idx, head_idx]
        attn_output = resid_after_mlp0 @ W_OV
        
        logits = resid2logits(attn_output)
        top_predictions = torch.topk(logits, dim=-1, k=5)
        percentage = (top_predictions == name_tokens).any(dim=-1).float().mean(dim=-1)
        

In [ ]:
class TensorProcessInitializer(ProcessGroupInitializer):
    def init_process_group(self):
        num_tensor_parallel_groups = self.world_size // self.tensor_parallel_size
        local_rank = None
        local_world_size = None
        
        for i in range(num_tensor_parallel_groups):
            ranks = list(range(
                i*self.tensor_parallel_size,
                (i+1)*self.tensor_parallel_size
            ))
            process_group = dist.new_group(ranks=ranks)
            
            if self.rank in ranks:
                local_rank = ranks.index(self.rank)
                local_world_size = len(ranks)
                
        return {
        
            }

In [ ]:
interactor, boundary, entity

In [18]:
import threading

In [19]:
event = threading.Event()

In [20]:
def run():
    print("waiting")
    event.wait()
    print("received")

In [21]:
thread = threading.Thread(target=run)

In [ ]:
thread.start()

In [ ]:
job selector > worker threads > pool watcher

In [ ]:
shared memory, file system, message passing

In [ ]:
scatter, broadcast, reduce, gather

In [ ]:
def by_row_parallelism(inputs, weights):
    inp_partition_size = inputs.shape[-1] // 2
    w_partition_size = weights.shape[-1] // 2
    
    inp1, inp2 = inputs[:, :inp_partition_size], inputs[:, inp_partition_size:]
    w1, w2 = weights[:w_partition_size, :], weights[w_partition_size:, :]
    
    out1 = inp1 @ w1
    out2 = inp2 @ w2
    
    return out1 + out2

In [ ]:
microbatch n > microbatch n-1 ...

In [ ]:
data prefetching, lazy loading, memory mapping

In [ ]:
scatteer, reduce

In [ ]:
broadcast > gather > scatter > all-reduce

In [ ]:
broadcast, gather

In [ ]:
scatter > reduce > identity > gather

In [22]:
import threading

In [23]:
lock = threading.Lock()

In [27]:
def intervene_resid(resid, hook, feature, scale, position):
    feature /= feature.norm(dim=-1)
    feature_projection = resid[0, position] @ feature
    resid[0, position] -= scale * feature * feature_projection

In [26]:
from transformer_lens.utils import get_act_name
from functools import partial

In [25]:
hook_name = get_act_name("resid_post", layer_idx)
hook_func = partial(
    intervene_resid,
    position=2,
    feature=target_feature,
    scale=scale_factor
)

In [ ]:
logits, _ = model.run_with_cache(
    past_moves,
    fwd_hooks=[hook_name, hook_func]
)

In [ ]:
def convert_to_distribution(states):
    q_values = q_function(states)
    return torch.exp(q_values)/q_values.sum()

In [ ]:
discount_returns = discounted_return_at_each_timestep(rewards)

In [ ]:
total_loss = 0

for discount_return, prob in zip(discount_returns, selected_action_probs):
    total_loss += discount_return * -prob.log(0